 # Build lookup tables for sliding rook-like and bishop-like attacks
 
 ## We can precalculate sliding attacks
Some pieces can perform sliding moves. They can move (and capture) until they hit another piece, traversing every squares (for rooks) or every other square (for bishops) or both (for queens). This is a little intensive to calculate at runtime. We can save time by precalculating and caching these in a map.

The input is a 20-bit integer, with the first 4 bits representing the location of the sliding piece and the remaining 16 bits representing occupancy. We can use any slow algorithm we wish to calculate the set of possible moves for the sliding piece, and finally we return a 16-bit integer representing that set. 

## It's not as much space as you'd think

We need to do this for every permutation of location, occupancy, and move type. This seems like a lot but upon calculation, it is very feasible:

Given `n` pieces on the board, the number of possible occupancies is at most ${16 \choose n}$, as there are 16 spots on the board and we are choosing `n` of them to put pieces on. (The real number is smaller, because there are occupancies that may be unreachable in a real game, but it's almost impossible to figure that out ahead of time.) For each occupancy, there are `n` possible locations for the sliding piece, and two move types to precalculate (rook-like and bishop-like). Thus we have:

$$2n * {16 \choose n}$$

permutations, given `n` pieces. We now need to sum over all possible values of `n`. The lower bound of `n` is 3 (two kings and a sliding piece) and the upper bound of `n` is 12 (the number of starting pieces). So we have:

$$\sum_{n=3}^{12} 2n * {16 \choose n}$$

total permutations to look at, which comes out to `1029632`. So for a mapping of a measly ~1M 20-bit keys to two 16-bit values each, we can look up sliding piece attacks almost instantly with only bitwise operations. That's only ~6.7 MB of data — not bad! 

## A clarifying example

Here is an example to clarify. Let us consider the black queen on square 6 in the following position:

`♔..♘.♕....♜♞...♚`

The location, square 6, can be represented in binary as:

`0110`

and the occupancy of the board can be represented in binary as:

`1001010000110001`

so, putting these together, the input is the 20-bit integer:

`01101001010000110001`

which, incidentally, corresponds to `431153` in decimal.

Let us consider the rook-like attack first. `x` represents attackable spaces:

`♔..xx♕xxxxx♞...♚`

which corresponds to the moveset:

`0001101111100000`

equivalent to the decimal number `7136`. Similarly for the bishop-like attack we have:

`♔..x.♕.x.x♜x...♚`

which corresponds to the moveset:

`0001000101010000`

equivalent to the decimal number `4432`. We can OR them to get the total moveset for the queen:

`0001101111110000`

equivalent to the decimal number `7152`.

So if we store the mapping `431153 -> (7136, 4432)` (we can generate the queen mapping by ORing the rook and bishop mappings) then we can efficiently lookup movesets for any sliding piece in any position.

## But it says it attacks its own color piece! That is wrong?

**Note** that the occupancy does not take into account color — this algorithm (correctly) indicates that the black queen can capture the white rook and white knight and also (incorrectly) indicates that it can take the black knight as well. This is intentional — it is easily fixed in a subsequent step by ANDing the moveset with the complement of the black occupancy.

Black occupancy is:

`1001010000000000`

and its complement (indicating all non-black squares, comprising white squares and empty squares) is:

`0110101111111111`

and the AND of that in the total moveset `0001101111110000 & 0110101111111111` gets us:

`0000101111110000`

which is what we expect from the original position, repeated here:

`♔..♘.♕....♜♞...♚`

In [21]:
from more_itertools import distinct_permutations

# Get occupancies.
occupancies = []
for n_occupied in range(3, 12+1):
    l = "1" * n_occupied + "0" * (16 - n_occupied)
    occupancies.extend(list(distinct_permutations(l)))
occupancies = ["".join(o) for o in occupancies]

In [81]:
# Traverse occupancy by interval and return valid moveset.
def get_moveset_from_occupancy(occupancy, pointer, interval):
    moveset = ["0"] * 16
    while True:
        pointer += interval
        if (0 <= pointer < 16):
            if occupancy[pointer] == "0":
                moveset[pointer] = "1"
            else:
                moveset[pointer] = "1"
                break
        else:
            break
    return int("".join(moveset), 2)  # turn binary string into int.
            

# Define function to make map.
def get_map_from_occupancy(occupancy):
    # Define mapping.
    m = []
    
    # Get indices of pieces.
    indices = set()
    for i,c in enumerate(occupancy):
        if c == "1":
            indices.add(i)
            
    # For each index, test sliding moveset.
    for i in indices:
        # Get input string.
        l = format(i, '04b')
        k = l + occupancy
        k = int(k, 2)  # turn binary string into int.
        
        # Get rook moveset.
        rook_left = get_moveset_from_occupancy(occupancy, i, -1)
        rook_right = get_moveset_from_occupancy(occupancy, i, 1)
        rook_total = rook_left | rook_right
#         print("R", i, format(rook_total,'016b'))
        
        # Get bishop moveset.
        bishop_left = get_moveset_from_occupancy(occupancy, i, -2)
        bishop_right = get_moveset_from_occupancy(occupancy, i, 2)
        bishop_total = bishop_left | bishop_right
#         print("B", i, format(bishop_total,'016b'))
        
        m.append(" ".join([str(s) for s in [k, rook_total, bishop_total]]))
    
    return(m)
    

mappings = []
for occupancy in occupancies:
    mappings.extend(get_map_from_occupancy(occupancy))
mappings = sorted(mappings, key=lambda x: int(x.split(" ")[0]))
    
with open('mapping.txt', 'w+') as f:
    f.writelines("\n".join(mappings))
    
print(len(mappings))
print(mappings[:25])

514816
['32771 32766 10922', '32773 32764 10922', '32774 32764 10922', '32775 32764 10922', '32777 32760 10920', '32778 32760 10920', '32779 32760 10920', '32780 32760 10920', '32781 32760 10920', '32782 32760 10920', '32783 32760 10920', '32785 32752 10922', '32786 32752 10922', '32787 32752 10922', '32788 32752 10922', '32789 32752 10922', '32790 32752 10922', '32791 32752 10922', '32792 32752 10920', '32793 32752 10920', '32794 32752 10920', '32795 32752 10920', '32796 32752 10920', '32797 32752 10920', '32798 32752 10920']


In [82]:
max_k = max((int(m.split(" ")[0]) for m in mappings))
max_rook_total = max((int(m.split(" ")[1]) for m in mappings))
max_bishop_total = max((int(m.split(" ")[2]) for m in mappings))

print(max_k, max_rook_total, max_bishop_total)

1048545 65533 43688


# Position expression

We can express positions as 
`halfmove     board`
`[0000 0000] [0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000]`

In [10]:
# C    COLOR
#  M   MULTIPLE MOVES
#   R  ROOK MOVES
#    B BISHOP MOVES
# 0000 empty
# 0001 pawn (white)
# 0010 knight (white)
# 0010 king (white)
# 0100 UNUSED
# 0101 bishop (white)
# 0110 rook (white)
# 0111 queen (white)
# 1000 UNUSED
# 1001 pawn (black)
# 1010 knight (black)
# 1011 king (black)
# 1100 UNUSED
# 1101 bishop (black)
# 1110 rook (black)
# 1111 queen (black)

# 00 empty
# 10 white
# 11 black

# Building a lookup table for all kinds of attacks
We make this as:

`[0000] [0000] [0000 0000 0000 0000]`

`[nibble] [location] [occupancy]`

In [124]:
# Define piece to nibble mapping.
piece_to_byte_map = {
    "P": 1,
    "N": 2,
    "B": 5,
    "R": 6,
    "Q": 7,
    "K": 3,
    "p": 9
}

byte_to_piece_map = {
    1: "P",
    2: "N",
    5: "B",
    6: "R",
    7: "Q",
    3: "K",
    9: "p"
}

In [109]:
from more_itertools import distinct_permutations

# Get list of every possible occupancy.
occupancies = []
for n_occupied in range(2, 12+1):
    l = "1" * n_occupied + "0" * (16 - n_occupied)
    occupancies.extend(list(distinct_permutations(l)))
occupancies = ["".join(o) for o in occupancies]

len(occupancies)

64822

In [110]:
# Traverse occupancy by intervals and return valid moveset.
# Intervals are a (int, boolean) tuple representing the interval and whether
# to repeat it.
def get_moveset_from_occupancy_and_intervals(occupancy, pointer, intervals):
    moveset = ["0"] * 16
    for interval in intervals:
        p = pointer
        while True:
            p += interval[0]
            if (0 <= p < 16):
                if occupancy[p] == "0":
                    moveset[p] = "1"
                else:
                    moveset[p] = "1"
                    break
            else:
                break
            if not interval[1]:  # if we don't want to repeat it, then end.
                break
    return int("".join(moveset), 2)  # turn binary string into int.


# Define function to make map.
def get_map(occupancies):
    # Define mapping.
    m = []

    # Iterate over occupancies.
    for occupancy in occupancies:
        # Get indices of pieces.
        indices = set()
        for i,c in enumerate(occupancy):
            if c == "1":
                indices.add(i)

        # For each occupancy, test every position.
        for i in indices:
            # Get index as nibble.
            index_nibble = format(i, '04b')

            # For every position, test every piece.
            for p, pb in piece_to_byte_map.items():
                # Get piece as nibble.
                piece_nibble = format(pb, '04b')

                # Combine into full binary string, then turn into int.
                binary_string = piece_nibble + index_nibble + occupancy
                binary = int(binary_string, 2)

                # Get moveset based on piece type.
                if p == "P":
                    if i == 5:
                        intervals = [(1, False), (2, False)]
                    else:
                        intervals = [(1, False)]
                elif p == "N":
                    intervals = [(-3, False), (-2, False), (2, False), (3, False)]
                elif p == "B":
                    intervals = [(-2, True), (2, True)]
                elif p == "R":
                    intervals = [(-1, True), (1, True)]
                elif p == "Q":
                    intervals = [(-2, True), (-1, True), (1, True), (2, True)]
                elif p == "K":
                    intervals = [(-1, False), (1, False)]
                elif p == "p": # special case for black pawn, which goes the other way.
                    if i == 10:
                        intervals = [(-2, False), (-1, False)]
                    else:
                        intervals = [(-1, False)]

                moveset = get_moveset_from_occupancy_and_intervals(occupancy, i, intervals)
                m.append((binary, moveset))
    
    return(m)

In [132]:
mappings = get_map(occupancies)

[(int(format(k, '016b').zfill(24)[4:8], 2),
  byte_to_piece_map[int(format(k, '016b').zfill(24)[:4], 2)],
  format(k, '016b').zfill(24)[8:], 
  format(v, '016b')) 
 for (k,v) in mappings[:10]]

[(14, 'P', '0000000000000011', '0000000000000001'),
 (14, 'N', '0000000000000011', '0000000000011000'),
 (14, 'B', '0000000000000011', '1010101010101000'),
 (14, 'R', '0000000000000011', '1111111111111101'),
 (14, 'Q', '0000000000000011', '1111111111111101'),
 (14, 'K', '0000000000000011', '0000000000000101'),
 (14, 'p', '0000000000000011', '0000000000000100'),
 (15, 'P', '0000000000000011', '0000000000000000'),
 (15, 'N', '0000000000000011', '0000000000001100'),
 (15, 'B', '0000000000000011', '0101010101010100')]

In [133]:
mappings[0]

(1966083, 1)

In [135]:
mappings = sorted(mappings, key=lambda x: x[0])
mappings = [" ".join((str(i) for i in m)) for m in mappings]
mappings[0]

'1081345 16384'

In [136]:
with open('mapping.txt', 'w+') as f:
    f.writelines("\n".join(mappings))

# Old version

In [ ]:
# Traverse occupancy by interval and return valid moveset.
def get_moveset_from_occupancy(occupancy, pointer, interval):
    moveset = ["0"] * 16
    while True:
        pointer += interval
        if (0 <= pointer < 16):
            if occupancy[pointer] == "0":
                moveset[pointer] = "1"
            else:
                moveset[pointer] = "1"
                break
        else:
            break
    return int("".join(moveset), 2)  # turn binary string into int.


# Define function to make map.
def get_map_from_occupancy(occupancy):
    # Define mapping.
    m = []
    
    # Get indices of pieces.
    indices = set()
    for i,c in enumerate(occupancy):
        if c == "1":
            indices.add(i)
            
    # For each index, test sliding moveset.
    for i in indices:
        # Get input string.
        l = format(i, '04b')
        k = l + occupancy
        k = int(k, 2)  # turn binary string into int.
        
        # Get rook moveset.
        rook_left = get_moveset_from_occupancy(occupancy, i, -1)
        rook_right = get_moveset_from_occupancy(occupancy, i, 1)
        rook_total = rook_left | rook_right
#         print("R", i, format(rook_total,'016b'))
        
        # Get bishop moveset.
        bishop_left = get_moveset_from_occupancy(occupancy, i, -2)
        bishop_right = get_moveset_from_occupancy(occupancy, i, 2)
        bishop_total = bishop_left | bishop_right
#         print("B", i, format(bishop_total,'016b'))
        
        m.append(" ".join([str(s) for s in [k, rook_total, bishop_total]]))
    
    return(m)
    

mappings = []
for occupancy in occupancies:
    mappings.extend(get_map_from_occupancy(occupancy))
mappings = sorted(mappings, key=lambda x: int(x.split(" ")[0]))
    
with open('mapping.txt', 'w+') as f:
    f.writelines("\n".join(mappings))
    
print(len(mappings))
print(mappings[:25])